# Import/Download libs

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scrape WIKI data

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_postal = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url_postal)
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup)
table = soup.find('table')
#print(tables.prettify())

column_names=['Postalcode','Borough','Neighborhood']
df_PBN = pd.DataFrame(columns=column_names)

i_df = 0
for each_tr in table.find_all('tr'):
    for each_td in each_tr.find_all('td'):
        temp = each_td.find_all('b')
        temp_p = temp[0].text
        temp = each_td.find_all('i')
        temp_b = 'Not assigned'
        temp_n = 'Not assigned'
        
        if len(temp) == 1:
            df_PBN.loc[i_df] = [temp_p, temp_b, temp_n]
            #print(temp[0].text)
        else:
            temp = each_td.find_all('a')
            if len(temp) > 0:
                temp_b = temp[0].text
                temp_n = []
                for i in range(1, len(temp)):
                    temp_n.append(temp[i].text)
                if len(temp_n) > 0:
                    temp_n = ', '.join(temp_n)
                else:
                    temp_n = 'Not assigned'
                #print(temp[0].text)
                #print(temp)
            else:
                temp_b = 'Not assigned'
                temp_n = 'Not assigned'
        df_PBN.loc[i_df] = [temp_p, temp_b, temp_n]
        #print(df_PBN.loc[i_df])
        i_df += 1
        #print(df_PBN)
        #print(each_td)
   
    #pppp
df_PBN.head(10)
df_PBN = df_PBN[df_PBN['Borough'] != 'Not assigned']
for i in df_PBN.index:
    if df_PBN.loc[i, 'Neighborhood'] == 'Not assigned':
         df_PBN.loc[i, 'Neighborhood'] =  df_PBN.loc[i, 'Borough']
df_PBN.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Queen's Park


# Get geospatial data and merge

In [3]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_ll = pd.read_csv("Toronto_long_lat_data.csv")
df_ll.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df_ll.set_index("Postalcode")
df_PBN.set_index("Postalcode")
df_merge=pd.merge(df_PBN, df_ll)
df_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:

df_tstr = df_merge[df_merge['Borough'].str.contains('Toronto',regex=False)]
df_tstr

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
24,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Downtown Toronto,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, King",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront, Union Station, Toronto Islands",43.640816,-79.381752
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


# Show borough on map if their name have 'Toronto'

In [5]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_tstr['Latitude'],df_tstr['Longitude'],df_tstr['Borough'],df_tstr['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [6]:
k=5
toronto_clustering = df_tstr[['Latitude','Longitude']]
#print(toronto_clustering)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
#print(kmeans.labels_)
if 'Cluster Labels' not in df_tstr.columns:
    df_tstr.insert(0, 'Cluster Labels', kmeans.labels_)
df_tstr['Cluster Labels'] = kmeans.labels_
df_tstr.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,2,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
24,2,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [7]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
kclusters = 5
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tstr['Latitude'], df_tstr['Longitude'], df_tstr['Neighborhood'], df_tstr['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if str(cluster) == 'nan':
        continue
    #print('cluster:[%s]' % str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## The same analysis we did to the New York City data. But it seems I have exceeded my call limit to Foursquare API so I keep having <KeyError: 'groups'> problem.

# Necessary info

In [8]:
CLIENT_ID = 'CYRKJGO1UOMYCBDU5EUJKMYXPDGS0SKRHAKAYPG4UTM4SOOC' # your Foursquare ID
CLIENT_SECRET = 'EHQ5XZMCNN2ELZJN5RCEFMC5LOKEEJUCIDKCAY22FZSRC0T2' # your Foursquare Secret
ACCESS_TOKEN = 'EKQFKVI01XTEHXFUTUHURDWV5N31J5KBFJQ1YNQEDVD1GF1W'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CYRKJGO1UOMYCBDU5EUJKMYXPDGS0SKRHAKAYPG4UTM4SOOC
CLIENT_SECRET:EHQ5XZMCNN2ELZJN5RCEFMC5LOKEEJUCIDKCAY22FZSRC0T2


# Explore Neighborhoods in Toronto

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)
            
        # make the GET request
        #print(requests.get(url).json()["response"])
        #if 'groups' not in requests.get(url).json()["response"]:
        #    print(requests.get(url).json()["response"])
        #    continue
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print('results:', results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
toronto_venues = getNearbyVenues(names=df_merge['Neighborhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude']
                                  )

Parkwoods


KeyError: 'groups'

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

# Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

# Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    #break

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# Run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#print(len(kmeans.labels_))
#print(len(neighborhoods_venues_sorted.index))
#print(len(toronto_merged.index))

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merge

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head(10) # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if str(cluster) == 'nan':
        continue
    #print('cluster:[%s]' % str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]